<a href="https://colab.research.google.com/github/BryanMed/Sequences-Time-Series-and-Prediction/blob/master/Semana2CrearDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del dataset

En este notebook vamos a ver los pasos para construir nuestro dataset. En el caso de las series temporales utilizaremos una ventana de valores los cuales serán nuestras **features x**, el valor que seguiría de la ventana es llamado **label y** y que corresponde al valor que corresponde a la futura predicción.

Sin más, empezamos instalando la versión de tensorflow.

In [0]:
!pip install tf-nightly-2.0-preview

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0-dev20191002


Vamos a crear un dataset a partir de una lista que va del 0 al 9.

In [7]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
  print(val.numpy())

0
1
2
3
4
5
6
7
8
9


Ahora separaremos los datos en ventanas de 5 valores, recorriendo/avanzando un valor cada vez (shift=1). Es decir, nos dará el valor de los primeros 5 valores(0-4), después (1-5), (2-6), (3-7), (4-8), (5-9), y como ya no se acompletan valores de 5 dado que la serie llega hasta 9, pues el tamaño de las ventanas disminuye (6-9), (7-9), ... 

In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()


0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


A nosotros solo son interesan aquellos valores cuya ventana es de tamaño 5, es por ello que utilizaremos el drop_reminder, quedándonos sólamente los de window_size=5.

In [11]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


A tensorflow le gusta que los datos estén en formato numpy

In [12]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))

for window in dataset:
  print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


Ahora separamos los parámetros de los labels, es decir, tomaremos los primeros 4 valores de cada ventana como nuestros datos de entrada, y el último valor de la ventana como el label y.

$$[parametros] \; [label]$$


In [19]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x, y in dataset:
  print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


Para evitar crear una tendencia con valores consecutivos en las ventanas, lo que haremos será darle una "barajeada" a los datos (shuffle).

In [25]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x, y in dataset:
  print(x.numpy(), y.numpy())



[5 6 7 8] [9]
[1 2 3 4] [5]
[2 3 4 5] [6]
[0 1 2 3] [4]
[3 4 5 6] [7]
[4 5 6 7] [8]


Finalmente agruparemos los datos en grupos de dos en dos

In [27]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())


x =  [[4 5 6 7]
 [3 4 5 6]]
y =  [[8]
 [7]]
x =  [[0 1 2 3]
 [2 3 4 5]]
y =  [[4]
 [6]]
x =  [[5 6 7 8]
 [1 2 3 4]]
y =  [[9]
 [5]]
